In [1]:
!nvidia-smi

Wed Nov 18 09:14:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 42%   61C    P2   117W / 280W |  20898MiB / 24220MiB |     93%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 81%   85C    P2   122W / 280W |  19108MiB / 24220MiB |     90%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/home/ironman/abhishek/Fixtures_Data/architectures/ResNeSt/')
sys.path.append('/home/ironman/abhishek/Fixtures_Data/Helper/')

from dataloader import mydataset, create_prime_dict 
from trainer_steplr import train, test_classify, eval_classify
from Load_model import load

##ResneSt
from resnest import resnest50 

In [3]:
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:6


**Dataloading Scheme**

In [4]:
trainlist = 'Fixtures_Data_manual_clean_new_training_list_toys.txt'
validlist ='Fixtures_Data_manual_clean_new_validation_list_toys.txt'

In [5]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  3323


In [6]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 256, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 128, num_workers=16,pin_memory=True)

**Model Definition**

In [7]:
model = resnest50(pretrained=False, num_classes = 3323)

model = nn.DataParallel(model,device_ids=[6,7]).to(device)

In [8]:
# # Transfer Learning

# for param in model.module.parameters():
#     param.requires_grad = False

# fc_inputs = model.module.fc.in_features

# model.module.fc = nn.Sequential(
#     nn.Linear(fc_inputs, 4096),
#     nn.BatchNorm1d(4096),
#     nn.ReLU(),
#     nn.Linear(4096, 3323)
    
# )

# model.to(device)

**Hyperparameters**

In [8]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)


# Epochs
num_Epochs = 120

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 30, gamma = 0.1)

In [9]:
modelname = 'Manually_cleaned_ResneSt_Backbone_StepLR'
modelpath = '/home/ironman/abhishek/saved_model_checkpoints/Fixture_Data/' + modelname

In [ ]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

loss 8.208235669136048
loss 16.02940034866333
loss 23.742013220787047
loss 31.38312083721161
loss 38.96142046451568
loss 46.429004092216495
loss 53.757518444061276
loss 60.93231418609619
loss 67.94764394760132
loss 74.85010375022888
Epoch:  1
training loss =  7.286130602325874
Validation Loss: 6.5205	Top 1 Validation Accuracy: 0.0748	 Top 5 Validation Accuracy: 0.1833
loss 6.0524730253219605
loss 11.738695797920228
loss 17.050418004989623
loss 21.913743538856508
loss 26.330781235694886
loss 30.420586059093477
loss 34.0991897726059
loss 37.5237686252594
loss 40.681575565338136
loss 43.61386062860489
loss 46.393762772083285
loss 48.958796865940094
Epoch:  2
training loss =  3.9960717290498633
Validation Loss: 11.7448	Top 1 Validation Accuracy: 0.6520	 Top 5 Validation Accuracy: 0.7934
loss 2.3576130509376525
loss 4.579378541707992
loss 6.679397850036621
loss 8.844540667533874
loss 10.85089342236519
loss 12.764140846729278
loss 14.617655653953552
loss 16.417902895212173
loss 18.2100263953

loss 0.7133293116092682
loss 1.4345827549695969
loss 2.1692493480443953
loss 2.892299200296402
loss 3.6398313933610917
loss 4.404302456974984
loss 5.151383163928986
loss 5.917499977946282
loss 6.666275652647019
loss 7.432846544384956
loss 8.202435212731361
loss 8.975593870282173
Epoch:  21
training loss =  0.7496392900862776
Validation Loss: 0.7854	Top 1 Validation Accuracy: 0.8306	 Top 5 Validation Accuracy: 0.9434
loss 0.7043449288606644
loss 1.4201604336500169
loss 2.1543229764699934
loss 2.8914800667762757
loss 3.6155925571918486
loss 4.354450217485428
loss 5.122089567184449
loss 5.880118581056595
loss 6.614221313595772
loss 7.376273136734962
loss 8.128581950068474
loss 8.890742916464806
Epoch:  22
training loss =  0.7425270780662037
Validation Loss: 0.8013	Top 1 Validation Accuracy: 0.8291	 Top 5 Validation Accuracy: 0.9427
loss 0.7190331438183785
loss 1.4091010013222693
loss 2.135898888409138
loss 2.867555862367153
loss 3.6112861636281015
loss 4.3401279219985005
loss 5.0717811486

Epoch:  40
training loss =  0.35359022086030123
Validation Loss: 0.5908	Top 1 Validation Accuracy: 0.8690	 Top 5 Validation Accuracy: 0.9629
loss 0.3328297454118729
loss 0.6776468732953072
loss 1.0190051962435245
loss 1.370941113382578
loss 1.706476054340601
loss 2.0598842079937456
loss 2.412558596730232
loss 2.7675549383461475
loss 3.1108664833009243
loss 3.4509794969856737
loss 3.8087709787487984
loss 4.171236881762743
Epoch:  41
training loss =  0.34861451699317536
Validation Loss: 0.5904	Top 1 Validation Accuracy: 0.8684	 Top 5 Validation Accuracy: 0.9623
loss 0.3354285843670368
loss 0.6813696014881134
loss 1.0146240869164467
loss 1.350862067192793
loss 1.6922615034878254
loss 2.023040497303009
loss 2.359905386418104
loss 2.6968718737363817
loss 3.0382141134142877
loss 3.3844126516580584
loss 3.731585360467434
loss 4.079101528078318
Epoch:  42
training loss =  0.34074869028058663
Validation Loss: 0.5959	Top 1 Validation Accuracy: 0.8682	 Top 5 Validation Accuracy: 0.9626
loss 0.323

loss 0.27429587513208387
loss 0.5417367948591709
loss 0.8181756433844566
loss 1.0948349376022817
loss 1.3690995502471923
loss 1.648100586682558
loss 1.937012739032507
loss 2.2282167537510396
loss 2.521450538188219
loss 2.809432730525732
loss 3.103443102389574
loss 3.406583620458841
Epoch:  60
training loss =  0.2844264928433944
Validation Loss: 0.6391	Top 1 Validation Accuracy: 0.8627	 Top 5 Validation Accuracy: 0.9595
loss 0.2658661761879921
loss 0.5125337794423104
loss 0.7507953700423241
loss 0.9865349985659122
loss 1.2228460387885571
loss 1.4511549474298955
loss 1.6877000986039639
loss 1.920964868813753
loss 2.155397744476795
loss 2.3916261383891104
loss 2.6144912186264992
loss 2.8437700782716275
Epoch:  61
training loss =  0.23654854823768987
Validation Loss: 0.6213	Top 1 Validation Accuracy: 0.8681	 Top 5 Validation Accuracy: 0.9616
loss 0.2163565580546856
loss 0.4367173373699188
loss 0.6548137314617634
loss 0.8717883679270745
loss 1.0982508669793605
loss 1.3163243766129016
loss 1

loss 1.4687158693373203
loss 1.6514343725889922
loss 1.8352725874632596
loss 2.027314670160413
loss 2.2132607373595237
Epoch:  79
training loss =  0.1850440031061711
Validation Loss: 0.6440	Top 1 Validation Accuracy: 0.8663	 Top 5 Validation Accuracy: 0.9605
loss 0.18408130675554277
loss 0.36019249752163884
loss 0.537570830732584
loss 0.7275477135181427
loss 0.9098607420921325
loss 1.0926622205227614
loss 1.2747868447005748
loss 1.4615412192046642
loss 1.6485638216137886
loss 1.8399520496279
loss 2.0217085375636814
loss 2.203775792196393
Epoch:  80
training loss =  0.18370021295867636
Validation Loss: 0.6404	Top 1 Validation Accuracy: 0.8654	 Top 5 Validation Accuracy: 0.9606
loss 0.17815708935260774
loss 0.3632787498831749
loss 0.5444401015341281
loss 0.7287700163573027
loss 0.9072466999292373
loss 1.0849459578841925
loss 1.2659700413793324
loss 1.4493840727210046
loss 1.6278454046696424
loss 1.8106641104817391
loss 1.9916892804205417
loss 2.1792241372168064
Epoch:  81
training loss =

In [10]:
#### Load saved model from checkpoint  #####
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [12]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

Validation Loss: 0.6577	Top 1 Validation Accuracy: 0.8654
Accuracy:defaultdict(<class 'int'>, {'Top 1 Accuracy': 86.54392178672833, 'Top 5 Accuracy': 96.05128355238122, 'Top 10 Accuracy': 97.26604806369464, 'Top 20 Accuracy': 98.00954248748647, 'Top 30 Accuracy': 98.32860111816878, 'Top 50 Accuracy': 98.66814975265639, 'Top 100 Accuracy': 99.03404267775078})	
